In [ ]:
!pip install tensorflow tensorrt keras

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import re

In [3]:
# for inputing csv file as the data, skip this block if the dataframe is already loaded
df = pd.read_csv('./data/df_data_filter.csv')
df = df.drop(columns=['Unnamed: 0'])
wq = ['BOD', 'AmNi', 'wq', 'DO', 'Ecoli', 'FC', 'NitraNi', 'NitriNi', 'OrPh', 'pH', 'Sal', 'SDD', 'Si', 'SS', 'Temp', 'TIN', 'TKN', 'ToNi', 'ToPh', 'Tur', 'UnAm', 'VSS']

In [4]:
# Define train and test datasets

df_train = df[df['Image_Year'] <= 2019].drop(columns=['Image_Year']).copy()
df_test = df[df['Image_Year'] == 2020].drop(columns=['Image_Year']).copy()
X_train = df_train.drop(columns = wq)
X_test = df_test.drop(columns = wq)

std_scaler = StandardScaler()
std_scaler.set_output(transform='pandas')

# Transform the data (centering and scaling features)
df_train_scaled = std_scaler.fit_transform(df_train)
X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.fit_transform(X_test)

df_scaled = std_scaler.fit_transform(df)

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow as tf

df = pd.read_csv('./data/df_data_filter.csv')
df = df.drop(columns=['Unnamed: 0'])

print("Starting LSTM training...")

wq = ['Chla', 'SS', 'Tur']
n_splits = 5
seed = 1

LSTM_results = []

def create_model(input_shape, units=50, lr=0.001, decay=0.97, momentum=0.9, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape, kernel_initializer='glorot_uniform'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    
    optimizer = Adam(learning_rate=lr, decay=decay, beta_1=momentum)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

for wq_name in wq:
    print(f"\n{wq_name} Started")
    
    for first in range(4, 13):
        print(f"nvar {first}")
        print(f"Seed {seed}")
        
        # Define the TimeSeriesSplit
        tscv = TimeSeriesSplit(n_splits=n_splits)

        for i, (train_index, test_index) in enumerate(tscv.split(df)):
            df_train, df_test = df.iloc[train_index], df.iloc[test_index]
            X_train, X_test = df_train.drop(columns=wq), df_test.drop(columns=wq)
            Y_train, Y_test = df_train[wq_name], df_test[wq_name]

            # Fit scaler on training data and transform both training and test data
            scaler_X = StandardScaler()
            scaler_Y = StandardScaler()
            X_train_scaled = scaler_X.fit_transform(X_train)
            X_test_scaled = scaler_X.transform(X_test)
            Y_train_scaled = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1)).flatten()
            Y_test_scaled = scaler_Y.transform(Y_test.values.reshape(-1, 1)).flatten()

            print("\n--------------------------------------")
            print(f'Training split: {train_index[0]} to {train_index[-1]}, Testing split: {test_index[0]} to {test_index[-1]}')

            # Correlation-based feature selection
            c = df_train.corr().copy()
            c = c[wq_name][22:113]
            c = abs(c).sort_values(ascending=False)[0:first]
            var = c.index.tolist()

            X_train2 = X_train_scaled[:, [X_train.columns.get_loc(col) for col in var]]
            X_test2 = X_test_scaled[:, [X_test.columns.get_loc(col) for col in var]]

            # Reshape data for LSTM input
            X_train2 = X_train2.reshape((X_train2.shape[0], 1, X_train2.shape[1]))
            X_test2 = X_test2.reshape((X_test2.shape[0], 1, X_test2.shape[1]))

            # Create and fit the LSTM model
            model = create_model(input_shape=(X_train2.shape[1], X_train2.shape[2]), units=50, lr=0.001, decay=0.97, momentum=0.9, dropout_rate=0.2)
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            
            model.fit(X_train2, Y_train_scaled, validation_split=0.1, epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping])

            # Evaluate model on training data
            Y_train_pred_scaled = model.predict(X_train2)
            Y_train_pred_scaled[Y_train_pred_scaled < 0] = 0.0

            # Inverse transform the predictions and actual values
            Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_scaled.reshape(-1, 1)).flatten()
            Y_train_unscaled = scaler_Y.inverse_transform(Y_train_scaled.reshape(-1, 1)).flatten()

            corr_model = np.corrcoef(Y_train_unscaled, Y_train_pred)[0, 1]
            rmse = mean_squared_error(Y_train_unscaled, Y_train_pred, squared=False)
            mae = mean_absolute_error(Y_train_unscaled, Y_train_pred)
            smape = np.mean(2 * (np.abs(Y_train_pred - Y_train_unscaled)) / (np.abs(Y_train_unscaled) + np.abs(Y_train_pred)))
            r_squared = 1 - (np.sum((Y_train_unscaled - Y_train_pred) ** 2) / np.sum((Y_train_unscaled - np.mean(Y_train_unscaled)) ** 2))
            adjusted_r_squared = 1 - (1 - r_squared) * (len(Y_train) - 1) / (len(Y_train) - X_train2.shape[2] - 1)

            print(f'{wq_name}: units: {50}, best_score: {r_squared}, RMSE: {rmse}')

            # Test model on testing data
            Y_test_pred_scaled = model.predict(X_test2)
            Y_test_pred_scaled[Y_test_pred_scaled < 0] = 0.0

            # Inverse transform the predictions and actual values
            Y_test_pred = scaler_Y.inverse_transform(Y_test_pred_scaled.reshape(-1, 1)).flatten()
            Y_test_unscaled = scaler_Y.inverse_transform(Y_test_scaled.reshape(-1, 1)).flatten()

            r_squared_test = 1 - (np.sum((Y_test_unscaled - Y_test_pred) ** 2) / np.sum((Y_test_unscaled - np.mean(Y_test_unscaled)) ** 2))
            corr_test = np.corrcoef(Y_test_unscaled, Y_test_pred)[0, 1]
            rmse_test = mean_squared_error(Y_test_unscaled, Y_test_pred, squared=False)
            mae_test = mean_absolute_error(Y_test_unscaled, Y_test_pred)
            smape_test = np.mean(2 * (np.abs(Y_test_pred - Y_test_unscaled)) / (np.abs(Y_test_unscaled) + np.abs(Y_test_pred)))
            print(f'{wq_name}: r_squared_test: {r_squared_test}, rmse_test: {rmse_test}, smape_test: {smape_test}')

            # Store results in a DataFrame
            LSTM_df = pd.DataFrame({
                'WQ': [wq_name], 'nvar': [first], 'var': [var], 'random_state': [seed],
                'units': [50], 'best_score': [r_squared],
                'r2': [r_squared], 'adjusted_r2': [adjusted_r_squared], 'corr_model': [corr_model],
                'rmse': [rmse], 'mae': [mae], 'smape': [smape],
                'r2_test': [r_squared_test], 'corr_test': [corr_test],
                'rmse_test': [rmse_test], 'mae_test': [mae_test], 'smape_test': [smape_test],
                'train_index': [train_index], 'test_index': [test_index],
                'Y_train': [Y_train], 'Y_train_pred': [Y_train_pred],
                'Y_test': [Y_test], 'Y_test_pred': [Y_test_pred]
            })

            LSTM_results.append(LSTM_df)
            print("--------------------------------------\n")

        print(f"{wq_name} Finished")

print("All LSTM training finished.")

# Combine all results into a single DataFrame
LSTM_result = pd.concat(LSTM_results, ignore_index=True)

# Save results to CSV
LSTM_result.to_csv("data/LSTM_results.csv", index=False)

print("Results saved to data/LSTM_results.csv")

2024-06-25 20:40:00.894928: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-25 20:40:01.720465: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 20:40:03.963230: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Starting LSTM training...

Chla Started
nvar 4
Seed 1

--------------------------------------
Training split: 0 to 58, Testing split: 59 to 117
Epoch 1/100


/home/rohinsood/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/rohinsood/.local/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - loss: 0.9614 - val_loss: 0.0667
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.7027 - val_loss: 0.0638
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.0845 - val_loss: 0.0615
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1.2012 - val_loss: 0.0593
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1.1775 - val_loss: 0.0571
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.8644 - val_loss: 0.0547
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.1541 - val_loss: 0.0525
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 1.0523 - val_loss: 0.0502
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 1.1268 - val_loss: 0.0480
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.0931 - val_loss: 0.0458
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.5853 - val_loss: 0.0437
Epoch 12/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.6463 - val_loss: 0.0418


/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/rohinsood/.local/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will b

...

Epoch 1/100


/home/rohinsood/.local/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 1.1928 - val_loss: 1.4743
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8552 - val_loss: 1.4570
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7114 - val_loss: 1.4430
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6436 - val_loss: 1.4335
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5414 - val_loss: 1.4236
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9548 - val_loss: 1.4119
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6120 - val_loss: 1.3997
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5169 - val_loss: 1.3925
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5607 - val_loss: 1.3856
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7074 - val_loss: 1.3754
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5224 - val_loss: 1.3691
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4042 - val_loss: 1.3650
Epoch 13/100

/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Results saved to data/LSTM_results.csv


# Optimal Models

In [4]:
df = pd.read_csv("data/LSTM_results.csv")

# Assuming df is your DataFrame
filtered_df = df[df['WQ'] == 'Tur']

lowest_rmse_entry = filtered_df.loc[filtered_df['rmse_test'].idxmin()]

print('Lowest rmse_test value:', lowest_rmse_entry['test_index'])
print('Other properties:', lowest_rmse_entry)


Lowest rmse_test value: [295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312
 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330
 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348
 349 350 351 352 353]
Other properties: WQ                                                            Tur
nvar                                                            9
var             ['LH_B4B5B6', 'LH_B5B6B7', 'B5_3', 'B5_2', 'LH...
random_state                                                    1
units                                                          50
best_score                                               0.398206
r2                                                       0.398206
adjusted_r2                                              0.379202
corr_model                                               0.661353
rmse                                                     3.849565
mae                                                      